# Aprendizaje Automático - Intermedio

## Introducción

En este micro curso aumentarás tu experiencia de aprendizaje automático al aprender cómo:

+ abordar los tipos de datos que a menudo se encuentran en conjuntos de datos del mundo real (**valores faltantes**, **variables categóricas**),
+ diseñar **pipelines** para mejorar la calidad de su código de aprendizaje automático,
+ usar técnicas avanzadas para la validación del modelo (**validación cruzada**),
+ crear modelos de última generación que se utilizan ampliamente para ganar concursos de Kaggle (**XGBoost**), y
+ evitar errores comunes e importantes en la ciencia de datos (**leakage**).

En el camino, consolidaremos nuestro conocimiento completando un ejercicio práctico con datos del mundo real en cada nuevo tema. Los ejercicios prácticos utilizan datos de [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course), donde utilizaremos 79 variables explicativas diferentes (como el tipo de tejado, la cantidad de habitaciones y la cantidad de baños) para predecir los precios de las viviendas. ¡Medirás tu progreso enviando predicciones a esta competición y observando cómo sube tu posición en la [clasificación](https://www.kaggle.com/c/home-data-for-ml-course/leaderboard)!

![Housing Prices](./images/housing_price.png)

## Ejercicio

Como calentamiento, revisaremos algunos fundamentos del aprendizaje automático y presentaremos nuestros resultados iniciales a una competición de Kaggle.

### Configuración

Trabajaremos con los datos de [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course) para predecir los precios de las viviendas en Iowa utilizando 79 variables explicativas que describen (casi) todos los aspectos de las casas.  

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Ejecuta la siguiente celda de código para cargar las funciones de entrenamiento y validación en `X_train` y `X_valid`, junto con los objetivos de predicción en `y_train` e `y_valid`. Las características de prueba se cargan en `X_test`.

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carga los datos
X_full = pd.read_csv('./input/train.csv', index_col='Id')
X_test_full = pd.read_csv('./input/test.csv', index_col='Id')

# Obtiene el objetivo y los predictores
y = X_full.SalePrice
features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = X_full[features].copy()
X_test = X_test_full[features].copy()

# Separa los datos de validación a partir de los datos de entrenamiento
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Usaremos `head` para mostrar las primeras filas de los datos. Es una buena manera de obtener una visión general de los datos que utilizaremos en nuestro modelo de predicción de precios.

In [15]:
X_train.head()

,LotArea,YearBuilt,1stFlrSF,2ndFlrSF,FullBath,BedroomAbvGr,TotRmsAbvGrd
Id,,,,,,,
619,11694,2007,1828,0,2,3,9
871,6600,1962,894,0,1,2,5
93,13360,1921,964,0,1,2,5
818,13265,2002,1689,0,2,3,7
303,13704,2001,1541,0,2,3,6


### Paso 1: Evaluar varios modelos

El siguiente código define cinco modelos de bosque aleatorio diferentes.

In [16]:
from sklearn.ensemble import RandomForestRegressor

# Define los modelos
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

Para seleccionar el mejor modelo de los cinco, definimos a continuación la función `score_model ()`. Esta función devuelve el error absoluto medio (MAE) del conjunto de validación. Recuerde que el mejor modelo obtendrá el MAE más bajo.

In [17]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_model(model, X_t=X_train, X_v=X_valid, y_t=y_train, y_v=y_valid):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Modelo %d MAE: %d" % (i+1, mae))

Modelo 1 MAE: 24015
Modelo 2 MAE: 23740
Modelo 3 MAE: 23528
Modelo 4 MAE: 23996
Modelo 5 MAE: 23706


Usa los resultados anteriores para completar la línea a continuación. ¿Qué modelo es el mejor? Tu respuesta debe ser una de las siguientes: `model_1`, `model_2`, `model_3`, `model_4` o `model_5`.

In [18]:
best_model = model_3

### Paso 2: generar predicciones de prueba

Excelente. Sabes cómo evaluar lo que hace que un modelo sea preciso. Ahora es el momento de pasar por el proceso de modelado y hacer predicciones. En la línea a continuación, crea un modelo Random Forest con el nombre de variable `my_model`.

In [19]:
# Define el model
my_model = best_model

El siguiente código entrena el modelo con los datos de capacitación y validación, y luego genera predicciones de prueba que se guardan en un archivo CSV. ¡Estas predicciones de prueba se pueden enviar directamente a la competición!

In [21]:
# Entrena el modelo con los datos de entrenamiento
my_model.fit(X, y)

# Genera predicciones de prueba
preds_test = my_model.predict(X_test)

# Guarda las predicciones en el formato usado para las puntuaciones de la competición
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('./output/submission.csv', index=False)

### Paso 3: envía tus resultados

Una vez que has completado con éxito el Paso 2, ¡estás listo para enviar tus resultados a la tabla de clasificación! Primero, deberás unirte a la competición si aún no lo has hecho. Abre una nueva ventana haciendo clic en [este enlace] (https://www.kaggle.com/c/home-data-for-ml-course). Luego haz clic en el botón **Join Competition**.

![unirse a la imagen de la competencia](https://i.imgur.com/wLmFtH3.png)

A continuación, sigue las instrucciones a continuación:
- Comienza haciendo clic en el botón azul **COMMIT** en la esquina superior derecha de esta ventana. Esto generará una ventana emergente.
- Una vez que tu código haya terminado de ejecutarse, haz clic en el botón azul **Open Version** en la esquina superior derecha de la ventana emergente. Esto te lleva al modo de vista de la misma página. Deberás desplazarte hacia abajo para volver a estas instrucciones.
- Haz clic en la pestaña **Output** a la izquierda de la pantalla. Luego, haz clic en el botón **Submit to Competition** para enviar tus resultados a la tabla de clasificación.
- Si deseas seguir trabajando para mejorar tu rendimiento, selecciona el botón azul **Edit** en la esquina superior derecha de la pantalla. Luego puedes cambiar tu modelo y repetir el proceso.